In [2]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from functools import partial

def calculate_weighted_gpa():
    courses = []
    total_credits = 0
    total_weighted_score = 0
    output = widgets.Output()
    course_list_output = widgets.Output()
    summary_output = widgets.Output()
    
    def update_summary():
        """ Updates the total GPA and total credits displayed at the top. """
        with summary_output:
            clear_output()
            if total_credits == 0:
                display(widgets.HTML("<h2>Total Credits: 0 | Weighted GPA: N/A</h2>"))
            else:
                weighted_gpa = total_weighted_score / total_credits
                display(widgets.HTML(f"<h2>Total Credits: {total_credits:.2f} | Weighted GPA: {weighted_gpa:.2f}"))
    
    def add_course(_):
        nonlocal total_credits, total_weighted_score
        name = course_name.value.strip()
        
        try:
            credits = float(course_credits.value)
            grade = float(course_grade.value)
        except ValueError:
            with output:
                clear_output()
                print("Invalid input. Please enter numeric values for credits and grade.")
            return
        
        if not name:
            with output:
                clear_output()
                print("Please enter a course name.")
            return
        
        courses.append([name, credits, grade])
        total_credits += credits
        total_weighted_score += grade * credits
        
        with output:
            clear_output()
            print(f"Course '{name}' added successfully!")
        
        update_summary()
        display_courses()
    
    def display_courses():
        with course_list_output:
            clear_output()
            if not courses:
                print("No courses entered yet.")
                return
            
            print("\nCourses Entered:")
            for idx, (name, credits, grade) in enumerate(courses, 1):
                edit_button = widgets.Button(description="Edit", button_style='info', layout=widgets.Layout(width='60px'))
                remove_button = widgets.Button(description="Remove", button_style='danger', layout=widgets.Layout(width='80px'))
                
                edit_button.on_click(partial(edit_course, index=idx))
                remove_button.on_click(partial(remove_course, index=idx))
                
                display(widgets.HBox([
                    widgets.Label(f"{idx}. {name}", layout=widgets.Layout(width='200px')),
                    widgets.Label(f"Credits: {credits}", layout=widgets.Layout(width='80px')),
                    widgets.Label(f"Grade: {grade}", layout=widgets.Layout(width='80px')),
                    edit_button, remove_button
                ]))
    
    def edit_course(_, index):
        nonlocal total_credits, total_weighted_score

        def update_course(_):
            nonlocal total_weighted_score, total_credits
            try:
                new_credits = float(edit_credits_input.value)
                new_grade = float(edit_grade_input.value)

                # Remove old values
                total_weighted_score -= courses[index - 1][2] * courses[index - 1][1]
                total_credits -= courses[index - 1][1]

                # Update with new values
                courses[index - 1][1] = new_credits
                courses[index - 1][2] = new_grade

                total_weighted_score += new_grade * new_credits
                total_credits += new_credits

                with output:
                    clear_output()
                    print(f"Updated '{courses[index - 1][0]}' successfully!")
                
                update_summary()
                display_courses()
            except ValueError:
                with output:
                    clear_output()
                    print("Invalid input. Please enter numeric values.")

        edit_credits_input = widgets.FloatText(value=courses[index - 1][1], layout=widgets.Layout(width='100px'))
        edit_grade_input = widgets.FloatText(value=courses[index - 1][2], layout=widgets.Layout(width='100px'))
        update_button = widgets.Button(description='Update', button_style='primary')
        update_button.on_click(update_course)
        
        with output:
            clear_output()
            print(f"Editing {courses[index - 1][0]}:")
            display(widgets.HBox([widgets.Label("New Credits:", layout=widgets.Layout(width='100px')), edit_credits_input]))
            display(widgets.HBox([widgets.Label("New Grade:", layout=widgets.Layout(width='100px')), edit_grade_input]))
            display(update_button)
    
    def remove_course(_, index):
        nonlocal total_credits, total_weighted_score
        
        removed_course = courses.pop(index - 1)
        total_credits -= removed_course[1]
        total_weighted_score -= removed_course[2] * removed_course[1]
        
        with output:
            clear_output()
            print(f"Removed '{removed_course[0]}' successfully!")
        
        update_summary()
        display_courses()
    
    # Input Fields with consistent width
    input_width = '100px'
    
    course_name = widgets.Text(layout=widgets.Layout(width='200px'))
    course_credits = widgets.FloatText(layout=widgets.Layout(width=input_width))
    course_grade = widgets.FloatText(layout=widgets.Layout(width=input_width))
    
    add_button = widgets.Button(description='Add Course', button_style='success')
    add_button.on_click(add_course)

    # Display UI
    display(summary_output)  # Pin summary to the top
    update_summary()  # Initial display
    
    display(widgets.VBox([
        widgets.Label('Enter Course Details:', layout=widgets.Layout(margin='10px 0')),
        widgets.HBox([widgets.Label("Course:", layout=widgets.Layout(width='80px')), course_name]),
        widgets.HBox([widgets.Label("Credits:", layout=widgets.Layout(width='80px')), course_credits]),
        widgets.HBox([widgets.Label("Grade:", layout=widgets.Layout(width='80px')), course_grade]),
        add_button
    ]))

    display(course_list_output)
    display(output)

# Run the interactive GPA calculator
calculate_weighted_gpa()

Output()

Output()

Output()